You are given an integer array nums. The range of a subarray of nums is the difference between the largest and smallest element in the subarray.

Return the sum of all subarray ranges of nums.

A subarray is a contiguous non-empty sequence of elements within an array.

 

Example 1:

Input: nums = [1,2,3]
Output: 4
Explanation: The 6 subarrays of nums are the following:
[1], range = largest - smallest = 1 - 1 = 0 
[2], range = 2 - 2 = 0
[3], range = 3 - 3 = 0
[1,2], range = 2 - 1 = 1
[2,3], range = 3 - 2 = 1
[1,2,3], range = 3 - 1 = 2
So the sum of all ranges is 0 + 0 + 0 + 1 + 1 + 2 = 4.
Example 2:

Input: nums = [1,3,3]
Output: 4
Explanation: The 6 subarrays of nums are the following:
[1], range = largest - smallest = 1 - 1 = 0
[3], range = 3 - 3 = 0
[3], range = 3 - 3 = 0
[1,3], range = 3 - 1 = 2
[3,3], range = 3 - 3 = 0
[1,3,3], range = 3 - 1 = 2
So the sum of all ranges is 0 + 0 + 0 + 2 + 0 + 2 = 4.
Example 3:

Input: nums = [4,-2,-3,4,1]
Output: 59
Explanation: The sum of all subarray ranges of nums is 59.
 

Constraints:

1 <= nums.length <= 1000
-109 <= nums[i] <= 109
 

Follow-up: Could you find a solution with O(n) time complexity?

In [ ]:
# brute force:
# - get all the sum arrays and find hte sum of the range difference.
# O(n^2)[for loop] * O(n) [finding min and max]
# sc - O(1)
def get_all_subarrays(nums):
    n = len(nums)
    s = 0
    
    for i in range(n):
        for j in range(i, n):
            subarray = nums[i: j+1]
            s += max(subarray) - min(subarray)
    
    return s


In [2]:
get_all_subarrays([4,-2,-3,4,1])

59

Key Idea:
- Instead of looping over all subarrays, calculate how many times each element appears as a min or max.
- For each element nums[i], calculate:
- How many subarrays where nums[i] is the minimum
- How many subarrays where nums[i] is the maximum

total += (number of times it’s max - number of times it’s min) * nums[i]


- Use monotonic increasing stack to count subarrays where each element is the minimum.
- Use monotonic decreasing stack to count subarrays where each element is the maximum.
- Compute the contribution of each element.

In [ ]:
class Solution:
    def subArrayRanges(self, nums: list[int]) -> int:
        n = len(nums)

        def count_contributions(nums, compare):
            stack = []
            res = [0] * n

            # The current index i has a previous element that is still valid in monotonic order.
            prev_ = [-1] * n

            # The one which get poped by the next greater/smaller number to maintain the monotonic order.
            next_ = [n] * n

            for i in range(n):
                while stack and compare(nums[stack[-1]], nums[i]):
                    next_[stack.pop()] = i
                if stack:
                    prev_[i] = stack[-1]
                stack.append(i)

            print(next_)
            print(prev_)
            for i in range(n):
                left = i - prev_[i]
                right = next_[i] - i
                res[i] = left * right
            return res

        # Count contributions for max and min.
        # top < ele: so maintaining a decreasing order. 
        # max_count: how many subarrays each element is the maximum
        max_count = count_contributions(nums, lambda a, b: a < b)

        # top < ele: so maintaining a increasing order. 
        # min_count: how many subarrays each element is the minimum
        min_count = count_contributions(nums, lambda a, b: a > b)
        print(max_count)
        print(min_count)

        total = 0
        for i in range(n):
            total += nums[i] * (max_count[i] - min_count[i])

        return total


# tc - O(n)
# sc - O(n)

In [6]:
Solution().subArrayRanges([1,2,3])

[1, 2, 3]
[-1, -1, -1]
[3, 3, 3]
[-1, 0, 1]
[1, 2, 3]
[3, 2, 1]


4

Great — let’s break this down **clearly** with a full explanation of how the formula works:

---

## 🔢 Goal:

We want to calculate, for each element `nums[i]`, how many **subarrays** exist where it is the **minimum** (or maximum) element. This is helpful for problems like:

* 2104. **Sum of Subarray Ranges**
* 907. **Sum of Subarray Minimums**
* 1856. **Maximum Subarray Min-Product**

To do that, we compute:

```python
left = i - prev[i]
right = next_[i] - i
res[i] = left * right
```

Then:

```python
return res
```

---

## 📘 What `prev[i]` and `next_[i]` mean:

Let’s say you're calculating **where `nums[i]` is the minimum**.

* `prev[i]`: index of **previous element smaller** than `nums[i]`
* `next_[i]`: index of **next element smaller** than `nums[i]`

(If there is no smaller element, `prev[i] = -1`, `next_[i] = n`)

This gives you the **boundaries** where `nums[i]` is the **smallest**.

---

## 🧠 Why `left = i - prev[i]`?

Because:

* You can start a subarray from any index between `prev[i]+1` and `i`
* So total choices for starting point: `i - prev[i]`

Similarly:

## 🧠 Why `right = next_[i] - i`?

Because:

* You can end a subarray at any index between `i` and `next_[i]-1`
* So total choices for end point: `next_[i] - i`

---

### 🔁 Multiply to count total subarrays

```python
res[i] = left * right
```

This gives us:

> All subarrays where `nums[i]` is the **minimum** (or maximum, depending on how stack was built).

---

## 📊 Example:

`nums = [3, 1, 2]`

We'll compute subarrays where each element is the **minimum**.

From monotonic stack logic:

| i | nums\[i] | prev\[i] | next\_\[i] | left = i - prev\[i] | right = next\_\[i] - i | res\[i] = left × right |
| - | -------- | -------- | ---------- | ------------------- | ---------------------- | ---------------------- |
| 0 | 3        | -1       | 1          | 0 - (-1) = 1        | 1 - 0 = 1              | 1 × 1 = 1              |
| 1 | 1        | -1       | 3          | 1 - (-1) = 2        | 3 - 1 = 2              | 2 × 2 = 4              |
| 2 | 2        | 1        | 3          | 2 - 1 = 1           | 3 - 2 = 1              | 1 × 1 = 1              |

### ✅ So:

* 3 is min in 1 subarray
* 1 is min in 4 subarrays
* 2 is min in 1 subarray

---

## 🏁 Final return:

```python
return res  # [1, 4, 1]
```

You can now:

* Multiply these by `nums[i]` to get their total contribution as min
* Or subtract max and min contributions for range sum problems

---

Let me know if you want the **code version** that generates `prev`, `next_`, and `res`, or if you'd like to walk through it for **maximum values** instead.
